# Shows how to access Google's Gemini LLM

## Prerequisites

1. You need an API key to access the Gemini API.
    - See the documentation, "Get a Gemini API key", at: https://ai.google.dev/gemini-api/docs/api-key
2. Choose a Gemini model to use.
    - See the documentation, "Gemini models", at: https://ai.google.dev/gemini-api/docs/models/gemini
    - They are free (as of 2024-12-29), but they have different rate limits,
        token limits, and supported data types.
3. Choose an API.
    - `google-generativeai`
        - We use this API in this notebook.
        - https://pypi.org/project/google-generativeai/
        - https://ai.google.dev/gemini-api/docs/quickstart?lang=python
    - `google-genai`
        - https://pypi.org/project/google-genai/
        - https://googleapis.github.io/python-genai/
        - The documentation says, "Please do not use this SDK in production."

In [1]:
# List of possible models: https://ai.google.dev/gemini-api/docs/models/gemini
GEMINI_MODEL_NAME = "gemini-2.0-flash-exp"

# The name of the key that you have defined as one of:
#   - an environment variable;
#   - in a .env file; or
#   - as a Secret in Google Colab.
GEMINI_API_KEY_NAME = "GEMINI_API_KEY"

In [2]:
import dotenv
import os
import google.generativeai as genai
import json

## Load API Key

In [3]:
# Loads variables defined in '.env' file as environment variables.
#   override=False means that the value of the variable in the environment
#     takes precedence over the value of the variable in the '.env' file.
_ = dotenv.load_dotenv(override=False)
GEMINI_API_KEY_VALUE = os.environ.get(GEMINI_API_KEY_NAME)

In [4]:
# In Google Colab, you have two options:
# (1) Prompt the user for the key value:
# import getpass
# GEMINI_API_KEY_VALUE = getpass.getpass()
# -or-
# (2) Save your API Key in the Secrets, then read that value:
# from google.colab import userdata
# GEMINI_API_KEY_VALUE = userdata.get('GEMINI_API_KEY_NAME')

## Get a model that returns JSON

See:

- https://ai.google.dev/gemini-api/docs/quickstart?lang=python#make-first-request
- https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb
- https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#python
- https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerationConfig

In [5]:
genai.configure(api_key=GEMINI_API_KEY_VALUE)
generation_configuration = genai.GenerationConfig(
    temperature=1.0,
    candidate_count=1,
    max_output_tokens=1024,
    response_mime_type="application/json",
)
model = genai.GenerativeModel(
    GEMINI_MODEL_NAME, generation_config=generation_configuration
)

## Submit a prompt

In [6]:
prompt = """
You are a creative writer.
I want you to provide a few paragraphs describing a city from the United States in the 1920s,
using a detective noir style.
The city has a corrupt government.
Gangsters on the South Side are making a lot of money
from bootleg liquor during Prohibition.
Detective Harston Cooper is trying to help Mrs. Lucille Robinson
to find her missing husband, Roland Robinson.
Provide the response as JSON in this format:
{
  "city": {
    "name": ,
    "description": ,
    "atmosphere": ,
    "mood":
  }
}
"""

In [7]:
raw_response = model.generate_content(prompt)
raw_response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "{\n  \"city\": {\n    \"name\": \"New Chicago\",\n    \"description\": \"The city was a symphony of shadows and secrets, a concrete jungle where the glint of chrome on a speeding car could just as easily be a reflection of a gangster's pistol. The towering skyscrapers, symbols of ambition and progress, cast long, dark fingers over the streets, hiding the rot that festered beneath the surface. The air was thick with the smell of coal smoke and desperation, a constant reminder of the city's gritty underbelly.\",\n    \"atmosphere\": \"A perpetual twilight seemed to cling to the city, even in the midday sun. The sounds of jazz music spilled from the speakeasies, mingling with the rumble of the elevated train and the hushed conversations in dimly lit alleyways. 

In [8]:
response = json.loads(raw_response.text)
response

{'city': {'name': 'New Chicago',
  'description': "The city was a symphony of shadows and secrets, a concrete jungle where the glint of chrome on a speeding car could just as easily be a reflection of a gangster's pistol. The towering skyscrapers, symbols of ambition and progress, cast long, dark fingers over the streets, hiding the rot that festered beneath the surface. The air was thick with the smell of coal smoke and desperation, a constant reminder of the city's gritty underbelly.",
  'atmosphere': 'A perpetual twilight seemed to cling to the city, even in the midday sun. The sounds of jazz music spilled from the speakeasies, mingling with the rumble of the elevated train and the hushed conversations in dimly lit alleyways. The South Side pulsed with a dangerous energy, where the sweet smell of illicit liquor masked the bitter taste of corruption.',
  'mood': 'The mood was a cocktail of unease and anticipation, a feeling that anything could happen at any moment. Every corner held 

In [9]:
response["city"]["name"]

'New Chicago'

In [10]:
response["city"]["description"]

"The city was a symphony of shadows and secrets, a concrete jungle where the glint of chrome on a speeding car could just as easily be a reflection of a gangster's pistol. The towering skyscrapers, symbols of ambition and progress, cast long, dark fingers over the streets, hiding the rot that festered beneath the surface. The air was thick with the smell of coal smoke and desperation, a constant reminder of the city's gritty underbelly."